In [21]:
from transformer import Transformer # this is the transformer.py file
import torch
import numpy as np

In [23]:
english_file="train.en"
hindi_file="train.hi"

START_TOKEN = '<START>'
PADDING_TOKEN = '<PAD>'
END_TOKEN = '<END>'

english_vocabulary=[START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@', 
                        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 
                        'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 
                        'Y', 'Z',
                        '[', '\\', ']', '^', '_', '`', 
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]


In [25]:
devanagari_chars = [chr(code) for code in range(0x0900, 0x0980)]

extra_tokens = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@','[', '\\', ']', '^', '_', '`','{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

hindi_vocabulary = devanagari_chars + extra_tokens

# for ch in extra_tokens:
#     if ch not in hindi_vocabulary:
#         hindi_vocabulary.append(ch)

# print("Total characters in Hindi vocabulary:", len(hindi_vocabulary))
# for ch in hindi_vocabulary:
#     print(repr(ch), end=' ')

In [27]:
len(english_vocabulary), len(hindi_vocabulary)

(97, 173)

In [29]:
index_to_hindi= {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [31]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(hindi_file, 'r') as file:
    hindi_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
hindi_sentences = hindi_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]

In [32]:
english_sentences[:10]

['in reply, pakistan got off to a solid start.',
 'the european union has seven principal decision-making bodies, its institutions: the european parliament, the european council, the council of the european union, the european commission, the court of justice of the european union, the european central bank and the european court of auditors.',
 'the congress leader represents sivaganga lok sabha segment from tamil nadu.',
 'prompt the user about connection attempts',
 'further, the minister announced that deposit insurance and credit guarantee corporation (dicgc) has been permitted to increase deposit insurance coverage to rs.',
 'therefore, brothers, be more diligent to make your calling and election sure. for if you do these things, you will never stumble.',
 'the review committee meeting chaired by the district judges will be attended by collectors, sps, superintendents of jails and secretary of district legal services authority.',
 'police is present on the spot.',
 'prime ministe

In [33]:
hindi_sentences[:10]

['जिसके जवाब में पाक ने अच्छी शुरुआत की थी.',
 'यूरोपीय संघ के महत्वपूर्ण संस्थानों में यूरोपियन कमीशन, यूरोपीय संसद, यूरोपीय संघ परिषद, यूरोपीय न्यायलय एवं यूरोपियन सेंट्रल बैंक इत्यादि शामिल हैं।',
 'कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.',
 'संबंधन प्रयास के बारे में उपयोक्ता को प्रांप्ट करें',
 'वित्त मंत्री ने घोषणा कि जमा बीमा और ऋण गारंटी निगम (डीआईसीजीसी) को जमा राशि बीमा का दायरा, जो इस समय 1 लाख रुपये है उसे बढ़ाकर प्रति जमाकर्ता 5 लाख रुपये करने की अनुमति प्रदान कर दी गई है।',
 'इस कारण हे भाइयों, अपने बुलाए जाने, और चुन लिये जाने को सिद्ध करने का भली भांति यत्न करते जाओ, क्योंकि यदि ऐसा करोगे, तो कभी भी ठोकर न खाओगे।',
 'समिति में जिला एवं सत्र न्यायाधीश, कलेक्टर, पुलिस अधीक्षक व जिला विधिक सेवा प्राधिकरण के सचिव शामिल रहते हैं।',
 'पुलिस मौके पर मौजूद है।',
 'जम्मू एवं कश्मीर विधानसभा के लिए दूसरे चरण के तहत होने वाले मतदान के लिए प्रधानमंत्री नरेंद्र मोदी भारतीय जनता पार्टी (भाजपा) के पक्ष में शुक्रवार को यहां रैलियां करेंगे।',
 'और 305

In [34]:
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Hindi: {np.percentile([len(x) for x in hindi_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )
     

97th percentile length Hindi: 260.0
97th percentile length English: 267.0


In [35]:
max_sequence_length = 270

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(hindi_sentences)):
    hindi_sentence, english_sentence = hindi_sentences[index], english_sentences[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")


Number of sentences: 200000
Number of valid sentences: 164565


In [36]:
hindi_sentences = [hindi_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]


In [37]:
hin_vocab_size = len(hindi_vocabulary)
hin_vocab_size

173

In [38]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 270
hin_vocab_size = len(hindi_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          hin_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)
     

In [39]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(97, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [40]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.hindi_sentences[idx]

In [41]:
dataset= TextDataset (english_sentences, hindi_sentences)

In [42]:
len(dataset)

164565

In [47]:
dataset[:5]

(['in reply, pakistan got off to a solid start.',
  'the congress leader represents sivaganga lok sabha segment from tamil nadu.',
  'prompt the user about connection attempts',
  'further, the minister announced that deposit insurance and credit guarantee corporation (dicgc) has been permitted to increase deposit insurance coverage to rs.',
  'therefore, brothers, be more diligent to make your calling and election sure. for if you do these things, you will never stumble.'],
 ['जिसके जवाब में पाक ने अच्छी शुरुआत की थी.',
  'कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.',
  'संबंधन प्रयास के बारे में उपयोक्ता को प्रांप्ट करें',
  'वित्त मंत्री ने घोषणा कि जमा बीमा और ऋण गारंटी निगम (डीआईसीजीसी) को जमा राशि बीमा का दायरा, जो इस समय 1 लाख रुपये है उसे बढ़ाकर प्रति जमाकर्ता 5 लाख रुपये करने की अनुमति प्रदान कर दी गई है।',
  'इस कारण हे भाइयों, अपने बुलाए जाने, और चुन लिये जाने को सिद्ध करने का भली भांति यत्न करते जाओ, क्योंकि यदि ऐसा करोगे, तो कभी भी ठोकर न खाओ

In [57]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('in reply, pakistan got off to a solid start.', 'the congress leader represents sivaganga lok sabha segment from tamil nadu.', 'prompt the user about connection attempts', 'further, the minister announced that deposit insurance and credit guarantee corporation (dicgc) has been permitted to increase deposit insurance coverage to rs.', 'therefore, brothers, be more diligent to make your calling and election sure. for if you do these things, you will never stumble.', 'the review committee meeting chaired by the district judges will be attended by collectors, sps, superintendents of jails and secretary of district legal services authority.', 'police is present on the spot.', 'prime minister narendra modi is slated to campaign for bjp in second phase of assembly elections in udhampur and poonch district of jammu and kashmir tomorrow', 'it has a battery backup of 3050mah.', '"similarly, babu raghuvansh prasad singh ji also worked for poor,"" he said in his address."', 'all 176 passengers d

In [59]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [60]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [52]:
# # quick sanity check on your tokenization
# from transformer import Transformer

# # make a tiny batch
# tiny_eng = english_sentences[:batch_size]
# tiny_hin = hindi_sentences[:batch_size]

# # tokenize on CPU
# enc_ids = transformer.encoder.sentence_embedding.batch_tokenize(tiny_eng,
#                                                                  start_token=False,
#                                                                  end_token=False)
# dec_ids = transformer.decoder.sentence_embedding.batch_tokenize(tiny_hin,
#                                                                  start_token=True,
#                                                                  end_token=True)

# print("encoder ids range:", enc_ids.min().item(), "to", enc_ids.max().item())
# print("decoder ids range:", dec_ids.min().item(), "to", dec_ids.max().item())
# print("english vocab size:", len(english_vocabulary))
# print("hindi vocab size:", len(hindi_vocabulary))


In [53]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, hin_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hin_batch)
        optim.zero_grad()
        hin_predictions = transformer(eng_batch,
                                     hin_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hin_batch, start_token=False, end_token=True)
        loss = criterian(
            hin_predictions.view(-1, hin_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hin_batch[0]}")
            hin_sentence_predicted = torch.argmax(hin_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in hin_sentence_predicted:
              if idx == hindi_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_hindi[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")


            transformer.eval()
            hin_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hin_sentence)
                predictions = transformer(eng_sentence,
                                          hin_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_hindi[next_token_index]
                hin_sentence = (hin_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (should we go to the mall?) : {hin_sentence}")
            print("-------------------------------------------")
     

Epoch 0
Iteration 0 : 5.944464206695557
English: in reply, pakistan got off to a solid start.
Hindi Translation: जिसके जवाब में पाक ने अच्छी शुरुआत की थी.
Hindi Prediction: अटॼऄ(((((((ऐऽऐऐऐॻॻ(तऐख़ॻऄ(((((((सऐऐऐॠ(ॠऐ(अ((तग(5अ(ऐ((स(((00ॖ0०((((((5((((((०(((0०((0००(ऐ(ु०((((5(((((((((०(०ञटञञञञञ((((((((((((ञ((००(ऐ(ञ(ञ(((((अ((((((((((ञ((((((((((((((अञ(अ.(अ5555(((((((((((((((8ऐ8(((((((((((ॖ.((((((((((((((((((((([अ(8एअ((((स(((((((=(((((55स((0(((ऴ((ॖअग
Evaluation translation (should we go to the mall?) : ('                                                             िि                                              ंंञञ                          क क  क क              ककककक                           88888ककककककक..888      ख़ख़88888888(((88   888080ऴससस(एएएससससऑऑऑ ङङङङङङङङ      ',)
-------------------------------------------
Iteration 100 : 3.4920310974121094
English: the supreme court referred the sabarimala matter to a larger bench.
Hindi Translation: सुप्रीम कोर्ट ने सबरीमाला केस को बड़ी बेंच को सौंप द

In [81]:
torch.save(transformer.state_dict(), "transformer_weights.pth")


In [63]:
transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          hin_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

transformer.load_state_dict(torch.load("transformer_weights.pth"))
transformer.to(device)
transformer.eval()


/tmp/ipykernel_4643/1005131344.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  transformer.load_state_dict(torch.load("transformer_weights.pth"))


Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(97, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [65]:
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  hin_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hin_sentence)
    predictions = transformer(eng_sentence,
                              hin_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_hindi[next_token_index]
    hin_sentence = (hin_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return hin_sentence[0]

In [67]:
translation = translate("what should we do when the day starts?")
print(translation)

क्या कहते हैं कि क्या है कि यह क्या है?<END>


In [69]:
translation = translate("noodles are the best")
print(translation)

निर्देश के लिए स्टेट करें<END>


In [71]:
translation = translate("this is the best thing ever")
print(translation)

यह स्थान की जा रही है।<END>
